# Predicting Customer Churn in Telecommunication Operators
Customer turnover refers to a decision made by the customer on the term of business relationship. Customer loyalty and customer turnover always add up to 100%. If a company has a 60% loyalty rate, then customer loss taxes are 40%. According to the 80/20 customer profitability rule, 20% of customers are generating 80% of revenue. Therefore, it is very important to predict the users who are likely to abandon the business relationship and the factors that affect how the customer's decisions.<br>
In this project, I predicted Customer Churn at a Telecommunications Operator using pyspark and python frameworks.

In [135]:
# importing libraries and frameworks
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark import sql
from pyspark.mllib.stat import Statistics 
from math import sqrt 
from pyspark.sql.functions import isnan, when, count, col

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

%matplotlib inline 

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Creating a spark session
spSession = SparkSession.builder.master("local").appName("local-SparkSQL").getOrCreate()

## Importing dataset

In [59]:
# Importing train dataset
trainRDD = sc.textFile("data/projeto4_telecom_treino.csv")
testRDD = sc.textFile("data/projeto4_telecom_teste.csv")
df_train = spark.read.csv("data/projeto4_telecom_treino.csv", header=True)
df_test = spark.read.csv("data/projeto4_telecom_teste.csv", header=True)

# Persisting on cache
trainRDD.cache()
testRDD.cache()

data/projeto4_telecom_teste.csv MapPartitionsRDD[123] at textFile at NativeMethodAccessorImpl.java:0

## Exploratory Analysis and Feature Engineering

In [60]:
# Checking dataset features
head = trainRDD.first()
print(len(head.split(",")))
df_train.head()

21


Row(id='1', state='KS', account_length='128', area_code='area_code_415', international_plan='no', voice_mail_plan='yes', number_vmail_messages='25', total_day_minutes='265.1', total_day_calls='110', total_day_charge='45.07', total_eve_minutes='197.4', total_eve_calls='99', total_eve_charge='16.78', total_night_minutes='244.7', total_night_calls='91', total_night_charge='11.01', total_intl_minutes='10', total_intl_calls='3', total_intl_charge='2.7', number_customer_service_calls='1', churn='no')

In [61]:
# Checking for missing values
df_train.select([count(when(isnan(c), c)).alias(c) for c in df_train.columns]).show()

+---+-----+--------------+---------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+-----------------------------+-----+
| id|state|account_length|area_code|international_plan|voice_mail_plan|number_vmail_messages|total_day_minutes|total_day_calls|total_day_charge|total_eve_minutes|total_eve_calls|total_eve_charge|total_night_minutes|total_night_calls|total_night_charge|total_intl_minutes|total_intl_calls|total_intl_charge|number_customer_service_calls|churn|
+---+-----+--------------+---------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+--------

In [66]:
print(df_train.count(),df_test.count())

3333 1667


In [185]:
# Compute numerical data summary statistics
df_train_num = df_train.select('account_length', 'number_vmail_messages', 'total_day_minutes', 
                               'total_day_calls', 'total_day_charge', 'total_eve_minutes', 
                               'total_eve_calls', 'total_eve_charge', 'total_night_minutes', 
                               'total_night_calls', 'total_night_charge', 'total_intl_minutes', 
                               'total_intl_calls', 'total_intl_charge', 'number_customer_service_calls')

df = df_train_num.toPandas().apply(pd.to_numeric)
df.dtypes


df_describe = pd.concat([df.describe().T,
              df.mad().rename('mad'),
              df.skew().rename('skew'),
              df.kurt().rename('kurt'),
              df.median().rename('median')
              ], axis=1).T

display(df_describe)

,account_length,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,101.064806,8.099010,179.775098,100.435644,30.562307,200.980348,100.114311,17.083540,200.872037,100.107711,9.039325,10.237294,4.479448,2.764581,1.562856
std,39.822106,13.688365,54.467389,20.069084,9.259435,50.713844,19.922625,4.310668,50.573847,19.568609,2.275873,2.791840,2.461214,0.753773,1.315491
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.200000,33.000000,1.040000,0.000000,0.000000,0.000000,0.000000
25%,74.000000,0.000000,143.700000,87.000000,24.430000,166.600000,87.000000,14.160000,167.000000,87.000000,7.520000,8.500000,3.000000,2.300000,1.000000
50%,101.000000,0.000000,179.400000,101.000000,30.500000,201.400000,100.000000,17.120000,201.200000,100.000000,9.050000,10.300000,4.000000,2.780000,1.000000
75%,127.000000,20.000000,216.400000,114.000000,36.790000,235.300000,114.000000,20.000000,235.300000,113.000000,10.590000,12.100000,6.000000,3.270000,2.000000
max,243.000000,51.000000,350.800000,165.000000,59.640000,363.700000,170.000000,30.910000,395.000000,175.000000,17.770000,20.000000,20.000000,5.400000,9.000000
mad,31.821440,11.719778,43.523455,15.944943,7.398914,40.469244,15.860332,3.439937,40.410387,15.690341,1.818555,2.184712,1.881093,0.589880,1.052532
skew,0.096606,1.264824,-0.029077,-0.111787,-0.029083,-0.023877,-0.055563,-0.023858,0.008921,0.032500,0.008886,-0.245136,1.321478,-0.245287,1.091359


In [187]:
# Compute categorical data summary statistics
df_train_cat = df_train.select('state', 'area_code', 'international_plan', 'voice_mail_plan', 'churn')
df_train_num.toPandas().describe()

,state,area_code,international_plan,voice_mail_plan,churn
count,3333,3333,3333,3333,3333
unique,51,3,2,2,2
top,WV,area_code_415,no,no,no
freq,106,1655,3010,2411,2850
